In [2]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from glob import glob
from xskillscore import crps_gaussian, rmse
from path_parameters import TRAIN_PATH, TEST_PATH, OUTPUT_PATH

In [3]:
data_path = TRAIN_PATH
test_path = TEST_PATH
output_path = OUTPUT_PATH

In [2]:
SECONDS_IN_YEAR = 60*60*24*365 #s
convert = lambda x: x * SECONDS_IN_YEAR * 1e-12 # kg -> Gt

# Only future scenarios for now
# TODO also test data here?
inputs = glob(data_path + "inputs_s*.nc")

def get_rmse(truth, pred):
    return np.sqrt(((truth-pred)**2))

def global_mean(ds):
    weights = np.cos(np.deg2rad(ds.latitude))
    return ds.weighted(weights).mean(['latitude', 'longitude'])


In [8]:
variables = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']

Y = xr.open_dataset(test_path + 'outputs_ssp245.nc').sel(time=slice(2050, 2100))

# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

gp_predictions = xr.merge([{v: xr.open_dataarray(output_path + "outputs_ssp245_predict_gp_{}.nc".format(v))} for v in variables]).sel(time=slice(2050, 2100))
gp_predictions_std = xr.merge([{v: xr.open_dataarray(output_path + "outputs_ssp245_predict_gp_{}_std.nc".format(v))} for v in variables]).sel(time=slice(2050, 2100))

In [24]:
for v in variables:
    print(crps_gaussian(Y[v], gp_predictions[v], gp_predictions_std[v], weights=np.cos(np.deg2rad(Y.lat))))

<xarray.DataArray 'tas' ()>
array(0.33017773)
Coordinates:
    quantile  float64 0.9
<xarray.DataArray 'diurnal_temperature_range' ()>
array(0.11699254)
Coordinates:
    quantile  float64 0.9
<xarray.DataArray 'pr' ()>
array(0.42198426)
Coordinates:
    quantile  float64 0.9
<xarray.DataArray 'pr90' ()>
array(1.26591589)
Coordinates:
    quantile  float64 0.9
